In [2]:
import minsearch

/workspaces/llm-workshop/01-intro/minsearch.py:9: UserWarning: Now minsearch is installable via pip: 'pip install minsearch'. Remove the downloaded file and re-install it with pip.
  warnings.warn(


In [3]:
import json


In [ ]:
#!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

Bad pipe message: %s [b'0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7\r\nHost: localhost:41625\r\nUs', b'-Agent: Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/133.']
Bad pipe message: %s [b'0.0 Safari/537.36 OPR/118.0.0.0\r\nAccept-Encodin']
Bad pipe message: %s [b' gzip, deflate, br, zstd\r\nAccept-Language: pl-PL,pl;q=0.9,en-US;q=0.8,en;q=0.7\r\nCache-Control: max-age']
Bad pipe message: %s [b'\r\nReferer: https://shiny-goggles-xrwpp4pxx97cv4j5.github.dev']
Bad pipe message: %s [b'\nX-Request-ID: de87eba384c6f2e642485e51c4589d2', b'\nX-Real-IP: 83.168.77.30\r\nX-Forwarded-Port: 443', b'X-Forwarded-']
Bad pipe message: %s [b'heme: https\r\nX-Original-URI: /\r\nX-Scheme: https\r\nsec-fetch-site: same-site\r\nsec-fe', b'h-mode: navigate\r\nsec-fetch-dest: document\r\nsec-ch-ua: "Not(A:Brand";v="99", "Opera";v="118", "Chromium";v="133"\r\ns', b'-ch-ua-mobile: ?0\r\nsec-ch-ua-platform: "Windows"\r\npriority:

In [4]:
with open('documents.json', 'rt') as f_in:
    docs_raw = json.load(f_in)

In [5]:
documents = []


In [6]:
for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [10]:
len(docs_raw)

3

In [15]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [16]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [17]:
#keywords = SELECT * WHERE course = 'data-engineering-zoomcamp';



In [18]:
q = 'the course has already started, can I still enroll?'


In [19]:
index.fit(documents)

In [20]:
from openai import OpenAI


In [40]:
OPENAI_API_KEY=""
client = OpenAI(api_key=OPENAI_API_KEY)


In [28]:
response = client.chat.completions.create(
    model='gpt-4o-mini',
    messages=[{"role": "user", "content": q}]
)

In [29]:
response

ChatCompletion(id='chatcmpl-Bgq4jqfomf803vYrrrfYmlvhuYoxd', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content="It depends on the specific course and institution's policies. Some courses may allow late enrollment, while others may not. I recommend checking with the course provider or institution directly for their enrollment guidelines and to see if there are any options available for joining the course after it has started.", refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1749549653, model='gpt-4o-mini-2024-07-18', object='chat.completion', service_tier='default', system_fingerprint='fp_34a54ae93c', usage=CompletionUsage(completion_tokens=55, prompt_tokens=18, total_tokens=73, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0

In [30]:
def search(query):
    boost = {'question': 3.0, 'section': 0.5}

    results = index.search(
        query=query,
        filter_dict={'course': 'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5
    )

    return results

In [31]:
search(q)

[{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 202

In [35]:
def build_prompt(query, search_results):
    prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

    context = ""
    
    for doc in search_results:
        context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"
    
    prompt = prompt_template.format(question=query, context=context).strip()
    return prompt

In [37]:
def llm(prompt):
    response = client.chat.completions.create(
        model='gpt-4o-mini',
        messages=[{"role": "user", "content": prompt}]
    )
    
    return response.choices[0].message.content

In [33]:
query = 'how do I run kafka?'

def rag(query):
    search_results = search(query)
    prompt = build_prompt(query, search_results)
    answer = llm(prompt)
    return answer

In [38]:
rag(query)

'To run Kafka components such as producers or consumers in Java, navigate to your project directory and execute the following command in the terminal:\n\n```\njava -cp build/libs/<jar_name>-1.0-SNAPSHOT.jar:out src/main/java/org/example/JsonProducer.java\n```\n\nMake sure to replace `<jar_name>` with the actual name of your JAR file.'

In [39]:
rag('the course has already started, can I still enroll?')

"Yes, you can still enroll in the course even after it has started. You are eligible to submit the homework, but keep in mind that there will be deadlines for turning in the final projects, so it's important not to leave everything to the last minute."